<a href="https://colab.research.google.com/github/UmarIgan/Machine-Learning/blob/master/microsoft_Phi1_5_wealth_alpaca_lora_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine Tuning with Microsoft phi
motivation: https://www.youtube.com/watch?v=R8CKx5yNEDo

# Installing Dependencies

- Installing libraries that will be used

In [1]:
! pip install accelerate transformers einops datasets peft bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 76.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 90.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.9 MB/s eta 0:00:00


- Login to huggingface for pushing model, get token wıth push feature**

In [16]:
from huggingface_hub import notebook_login
notebook_login()

# Imports

In [2]:
import torch
from datasets import load_dataset, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
import os

# Training&Fine-Tuning

- Good result seem with phi so i use it here.

In [3]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1_5", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

setting up config for [bitsandbytes](https://colab.research.google.com/drive/1b0GMx_Cp0huYVi6OOmciZ1smzkKYS7DP#scrollTo=L7hYn0WwEI68&line=1&uniqifier=1) for quantization.

In [4]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-1_5",
    device_map={"":0},
    trust_remote_code=True,
    quantization_config=bnb_config
)

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-1_5:
- configuration_mixformer_sequential.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-1_5:
- modeling_mixformer_sequential.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


- lora for accelerates the fine-tuning

In [7]:
config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["Wqkv", "out_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 4,718,592 || all params: 1,422,989,312 || trainable%: 0.3315971497613047


In [8]:
def add_prefix(example):#function for creating instructor
    example["text"] = ('question: ' + ' '.join([str(item) for item in example['instruction']]) + " answer: "+  example["output"])
    return example


In [6]:
def tokenize(sample):
    model_inps =  tokenizer(sample["text"], padding=True, truncation=True, max_length=512)
    return model_inps

In [9]:
data = load_dataset("gbharti/wealth-alpaca_lora",  split="train[:40000]")#we will left some data for testing fine tunde model
data =  data.map(add_prefix)
tokenized_data = data.map(tokenize, batched=True, desc="Tokenizing data", remove_columns=data.column_names)
tokenized_data

Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

Tokenizing data:   0%|          | 0/40000 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 40000
})

In [10]:
training_arguments = TrainingArguments(
        output_dir="phi-1_5-finetuned-wealth-alpaca_lora",
        per_device_train_batch_size=4,
        gradient_accumulation_steps=1,
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=100,
        max_steps=1000,
        num_train_epochs=1,
        push_to_hub=True
    )

In [11]:
trainer = Trainer(
    model=model,
    train_dataset=tokenized_data,
    args=training_arguments,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
trainer.train()

You're using a CodeGenTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
100,2.547600
200,2.525800
300,2.421800
400,2.412900
500,2.498500
600,2.380500
700,2.389000
800,2.483300
900,2.409700
1000,2.515500


TrainOutput(global_step=1000, training_loss=2.4584663543701173, metrics={'train_runtime': 372.993, 'train_samples_per_second': 10.724, 'train_steps_per_second': 2.681, 'total_flos': 8770631329972224.0, 'train_loss': 2.4584663543701173, 'epoch': 0.1})

In [12]:
trainer.push_to_hub()

'https://huggingface.co/umarigan/phi-1_5-finetuned-wealth-alpaca_lora/tree/main/'

# Saving
-adding base model

In [14]:
from peft import PeftModel
from transformers import AutoModelForCausalLM
import torch
model = AutoModelForCausalLM.from_pretrained("microsoft/phi-1_5", trust_remote_code=True, torch_dtype=torch.float32)
peft_model = PeftModel.from_pretrained(model, "umarigan/phi-1_5-finetuned-wealth-alpaca_lora", from_transformers=True)
model = peft_model.merge_and_unload()

MixFormerSequentialForCausalLM(
  (layers): Sequential(
    (0): Embedding(
      (wte): Embedding(51200, 2048)
      (drop): Dropout(p=0.0, inplace=False)
    )
    (1): ParallelBlock(
      (ln): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (resid_dropout): Dropout(p=0.0, inplace=False)
      (mixer): MHA(
        (rotary_emb): RotaryEmbedding()
        (Wqkv): Linear(in_features=2048, out_features=6144, bias=True)
        (out_proj): Linear(in_features=2048, out_features=2048, bias=True)
        (inner_attn): SelfAttention(
          (drop): Dropout(p=0.0, inplace=False)
        )
        (inner_cross_attn): CrossAttention(
          (drop): Dropout(p=0.0, inplace=False)
        )
      )
      (mlp): MLP(
        (fc1): Linear(in_features=2048, out_features=8192, bias=True)
        (fc2): Linear(in_features=8192, out_features=2048, bias=True)
        (act): NewGELUActivation()
      )
    )
    (2): ParallelBlock(
      (ln): LayerNorm((2048,), eps=1e-05, elementwis

In [18]:
model.push_to_hub("phi-1_5-finetuned-wealth-alpaca_lora_uu")

pytorch_model.bin:   0%|          | 0.00/5.67G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/umarigan/phi-1_5-finetuned-wealth-alpaca_lora_uu/commit/c254b34213c199ac6577c1ca17a446afd7241af1', commit_message='Upload MixFormerSequentialForCausalLM', commit_description='', oid='c254b34213c199ac6577c1ca17a446afd7241af1', pr_url=None, pr_revision=None, pr_num=None)

# Testing

In [19]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
model = AutoModelForCausalLM.from_pretrained("umarigan/phi-1_5-finetuned-wealth-alpaca_lora_uu", trust_remote_code=True, torch_dtype=torch.float32)
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1_5", trust_remote_code=True)

In [30]:
data_test = load_dataset("gbharti/wealth-alpaca_lora",  split="train[40000:]")#test data

In [36]:
data_test[131]#grab a question and as it to model

{'instruction': 'Generate a list of 5 books for the given genre.',
 'output': "1. The Martian by Andy Weir\n2. 1984 by George Orwell\n3. The Hitchhiker's Guide to the Galaxy by Douglas Adams\n4. Dune by Frank Herbert\n5. The Foundation Trilogy by Isaac Asimov",
 'input': 'Sci-Fi'}

In [24]:
inputs = tokenizer('''question: What do the terms par value, purchase price, call price, call date, and coupon rate mean in the context of bonds? answer: ''', return_tensors="pt", return_attention_mask=False)
outputs = model.generate(**inputs, max_length=1048)
text = tokenizer.batch_decode(outputs)[0]
text

'question: What do the terms par value, purchase price, call price, call date, and coupon rate mean in the context of bonds? answer: \nThe par value is the face value of the bond. The purchase price is the amount of money you paid to buy the bond. The call price is the amount of money you would get if you sold the bond. The call date is the date on which you can call the bond. The coupon rate is the percentage of the face value that you will receive each year. For example, if you buy a bond with a coupon rate of 5%, you will receive 5% of the face value each year. \nThe terms are used to describe the different aspects of a bond. The par value is the face value of the bond. The purchase price is the amount of money you paid to buy the bond. The call price is the amount of money you would get if you sold the bond. The call date is the date on which you can call the bond. The coupon rate is the percentage of the face value that you will receive each year. \nThe terms are used to describe 

In [28]:
import requests

# Replace 'YOUR_BOT_TOKEN' with the token you received from BotFather
BOT_TOKEN = "5290790328:AAFSSr2dI2nSJ3zyXE9_vV1sQhMDwCMfWC8"

# Replace 'YOUR_CHAT_ID' with your Telegram chat ID
# You can find your chat ID by starting a chat with the "userinfobot" on Telegram
CHAT_ID = '647615802'

def send_telegram_message(message):
    api_url = f'https://api.telegram.org/bot{BOT_TOKEN}/sendMessage'
    params = {
        'chat_id': CHAT_ID,
        'text': message,
    }

    response = requests.post(api_url, params=params)

    if response.status_code == 200:
        print('Message sent successfully!')
    else:
        print('Message sending failed.')

send_telegram_message(message_to_send)


Message sent successfully!
